In [1]:
import pdfplumber
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import re

In [2]:
def preprocess_image(image):
    """Preprocess image for better OCR accuracy."""
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return Image.fromarray(thresh)

In [5]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF, handling both text-based and scanned pages."""
    extracted_text = {}

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()

            if text and text.strip():
                extracted_text[f"Page {page_num+1}"] = text.strip()
            else:
                # OCR on the image if text extraction fails
                images = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1, poppler_path = poppler_path)
                ocr_text = ""
                
                for img in images:
                    processed_img = preprocess_image(img)  # Preprocess for better OCR
                    # Use '--psm 6' for better structure & '--oem 3' for LSTM OCR
                    ocr_text += pytesseract.image_to_string(processed_img, config="--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789.,")  

                extracted_text[f"Page {page_num+1}"] = ocr_text.strip()

    return extracted_text


In [6]:
# Usage Example
poppler_path = "/opt/homebrew/bin"
# Specify the path to Tesseract
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"

pdf_path = "PDF PUrchase jan-25.pdf"  # Change to your PDF file
extracted_text = extract_text_from_pdf(pdf_path)


# Optionally, save to a text file
with open("extracted_text.txt", "w", encoding="utf-8") as f:
    for page, text in extracted_text.items():
        f.write(f"{page}:\n{text}\n" + "-"*80 + "\n")

In [7]:
def extract_Mindmill_invoice_data(pdf_text):
    extracted_info = {}
    
    #define regex patterns for each required information
    patterns = {
        "Name of Supplier": r"Mindmill Software Limited",
        "Supplier's GSTN": r"GSTIN/UIN:\s*(\w+)",
        "Tax Invoice Number": r"Registered Office:\s*(\w+-\w+-\w+)",
        "Date of Invoice": r"Registered Office:.*?(\d{1,2}-[A-Za-z]{3}-\d{4})|Registered Office:.*?(\d{1,2}[A-Za-z]{3}-\d{4})",
       "Description of Goods/services": r"Description of Goods.*?\n(.*?)\nTotal Charges|Electricity charges for\s*([\w\d,]+)",
       "Total Taxable Value": r"Total Charges\s*(\d+\.?\d*)|Electricity charges for .*?\s+([\d,]+\.\d{2})\)?",
       "Central Tax": r"CGST ON RENTAL SERVICES@9%\s+\d+\s*%\s*([\d,]+\.\d{2})|CGST on electrcity expense@\s*\d+%\s*%\s*([\d,]+\.\d{2})",
       "State Tax": r"SGST ON RENTAL SERVICES\s+@\s*9%\s+\d+\s*%\s*([\d,]+\.\d{2})|SGST on electrcity expense@\s*\d+%\s*%\s*([\d,]+\.\d{2})",
       "Integrated Tax": r"Total\s*Amount Chargeable. *?(\d+\.?\d*)"
    }
    
    #extarcted the information using the defined patterns
    for key, pattern in patterns.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            groups = match.groups()  # Get all groups
            extracted_info[key] = (
                   groups[0] if groups and groups[0] is not None  # Pick first non-None group
                   else (groups[1] if len(groups) > 1 else match.group(0)))
        else:
            extracted_info[key] = 0
    #extracted_info['Total C'] = float(extracted_info.get('Total Taxable Value', 0)) + float(extracted_info.get('State Tax', 0)) + float(extracted_info.get('Central Tax', 0))
    return extracted_info

In [8]:
def extract_vifa_invoice_data(pdf_text):
    extracted_info = {}

    # Define regex patterns for VIFA invoice format
    patterns = {
        "Name of Supplier": r"VIFA Management Private Limited",
        "Supplier's GSTN": r"GSTIN:\s*([\w\d]+)",
        "Tax Invoice Number": r"INVOICE #\s*=\s*([\w/-]+)",
        "Date of Invoice": r"DATE:\s*(\d{4}/\d{2}/\d{2})",
        "Description of Goods/services": r"DESCRIPTION\s*(.*?)\s*Subtotal",
        "Total Taxable Value": r"Subtotal\s*OTHER COMMENTS IGST\d+%\s*([\d,]+\.\d{2})",
        "Central Tax": r"CGST\d+%\s*([\d,]+\.\d{2})",  # VIFA uses IGST, so CGST/SGST may not be present
        "State Tax": r"SGST\d+%\s*([\d,]+\.\d{2})",   # VIFA uses IGST, but included in case
        "Integrated Tax": r"IGST\d+%\s*([\d,]+\.\d{2})",
    }

    # Extract the information using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            groups = match.groups()
            extracted_info[key] = (
                groups[0] if groups and groups[0] is not None
                else (groups[1] if len(groups) > 1 else match.group(0))
            )
        else:
            extracted_info[key] = 0  # No match found
    #extracted_info['Total C'] = sum(float(extracted_info.get(key, 0) or 0) for key in ["Total Taxable Value", "State Tax", "Central Tax"])
    
    return extracted_info


In [12]:
def extract_slv_invoice_data(pdf_text):
    extracted_info = {}
    
    patterns = {
        "Name of Supplier": r"SLV SECURITY SERVICES PVT\. LTD\.",
        "Supplier's GSTN": r"GST No\s*:\s*(\d+[A-Z]+\d+[A-Z]+\d+)",
        "Tax Invoice Number": r"Invoice No\s*:\s*([A-Z0-9/-]+)",
        "Date of Invoice": r"Invoice Date\s*:\s*(\d{2}-[A-Za-z]{3}-\d{4})",
        "Description of Goods/services": r"Sr\.\s+Description / Details.*?\n(.*?)\nBank",
        "Total Taxable Value": r"Taxable Value\s*(\d+,?\d*\.?\d*)",
        "Central Tax": r"CGST\s*(\d+,?\d*\.?\d*)",
        "State Tax": r"SGST\s*(\d+,?\d*\.?\d*)",
        "Integrated Tax": r"Total GST Amount\s*(\d+,?\d*\.?\d*)",
    }
    for key, pattern in patterns.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            groups = match.groups()
            extracted_info[key] = (
                groups[0] if groups and groups[0] is not None
                else (groups[1] if len(groups) > 1 else match.group(0))
            )
        else:
            extracted_info[key] = 0
    
    return extracted_info

In [13]:
def extract_ernst_young_invoice_details(pdf_text):
    extracted_info = {}
    patterns = {
        "Name of Supplier": r"Ernst & Young LLP",
        "Supplier's GSTN": r"Our GSTIN No:\s*([\w\d]+)",
        "Tax Invoice Number": r"Invoice Number\s*([\w\d]+)",
        "Date of Invoice": r"Date:\s*([\dA-Za-z ]+)",
        "Description of Goods/services": r"(?s)(?<=Base Amount Tax Type Tax Rate Tax Amount Total Amount\s)(.*?)(?=\d{1,3}(?:,\d{3})*\.\d{2})",
        "Total Taxable Value": r"Total Invoice amount:\s*([\d,]+\.\d{2})",
        "Central Tax": "0",  # No CGST/SGST in this invoice
        "State Tax": "0",  # No CGST/SGST in this invoice
        "Integrated Tax": r"IGST\s+([\d,]+\.\d{2})"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            groups = match.groups()
            extracted_info[key] = (
                groups[0] if groups and groups[0] is not None
                else (groups[1] if len(groups) > 1 else match.group(0))
            )
        else:
            extracted_info[key] = 0
    
    return extracted_info

In [14]:
def extract_good_food_invoice_data(pdf_text):
    """
    Extracts relevant details from an Amazon tax invoice.
    """
    extracted_data = {}

    # Define patterns for each required field
    patterns = {
        "Name of Supplier": r"GOOD FOOD RETAIL PRIVATE LIMITED",
        "Supplier's GSTN": r"GST Registration No:\s*([\w\d]+)",
        "Tax Invoice Number": r"Invoice Number\s*:\s*([\w\d\-\/]+)",
        "Date of Invoice": r"Invoice Date\s*:\s*([\d\/\-]+)",
        "Description of Goods/services": r"Double Pagoda Chinklang Vinegar, 550ml|Vinegar",
        "Total Taxable Value": r"(\d+\.\d+)\s*\|\s*18%\s*IGST",  # Extract before 18% IGST
        "Central Tax": r"CGST\s*(\d+\.\d+)",  # If CGST exists
        "State Tax": r"SGST\s*(\d+\.\d+)",  # If SGST exists
        "Integrated Tax": r"IGST\s*\|?\s*(\d+\.\d+)",  # Extract IGST value
    }
    for key, pattern in patterns.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            groups = match.groups()
            extracted_info[key] = (
                groups[0] if groups and groups[0] is not None
                else (groups[1] if len(groups) > 1 else match.group(0))
            )
        else:
            extracted_info[key] = 0
    
    return extracted_info

In [15]:
all_extracted_info = []
for key, text in extracted_text.items():
    if "Mindmill Software Limited" in text:
        page_info = extract_Mindmill_invoice_data(text)
        page_info['Page No'] = key  # Add page number to extracted info
        all_extracted_info.append(page_info)  # Append to the list
    elif "VIFA Management Private Limited" in text:
        page_info = extract_vifa_invoice_data(text)
        page_info['Page No'] = key  # Add page number to extracted info
        all_extracted_info.append(page_info)  # Append to the list
    elif "SLV SECURITY SERVICES PVT. LTD." in text:
        page_info = extract_slv_invoice_data(text)
        page_info['Page No'] = key  # Add page number to extracted info
        all_extracted_info.append(page_info)  # Append to the list
    elif "Ernst & Young LLP" in text:
        page_info = extract_ernst_young_invoice_details(text)
        page_info['Page No'] = key  # Add page number to extracted info
        all_extracted_info.append(page_info)  # Append to the list
    elif "GOOD FOOD RETAIL PRIVATE LIMITED" in text:
        page_info = extract_good_food_invoice_data(text)
        page_info['Page No'] = key  # Add page number to extracted info
        all_extracted_info.append(page_info)  # Append to the list

In [16]:
# Convert to DataFrame
df = pd.DataFrame(all_extracted_info)

In [18]:
df['Total Taxable Value'] = df['Total Taxable Value'].str.replace(',','').astype(float)
df['Central Tax'] = df['Central Tax'].str.replace(',','').astype(float)
df['State Tax'] = df['State Tax'].str.replace(',','').astype(float)
df['Integrated Tax'] = df['Integrated Tax'].str.replace(',','').astype(float)

In [19]:
df['Total C'] = df[['Total Taxable Value', 'Central Tax', 'State Tax', 'Integrated Tax']].sum(axis = 1)

In [20]:
df

,Name of Supplier,Supplier's GSTN,Tax Invoice Number,Date of Invoice,Description of Goods/services,Total Taxable Value,Central Tax,State Tax,Integrated Tax,Page No,Total C
0,Mindmill Software Limited,09AAACM8388C1Z9,MSLRS24-25-292,1-Jan-2025,No.\n1 Building Rental - First Floor - (New Ar...,300000.00,27000.00,27000.00,NaN,Page 1,354000.00
1,SLV SECURITY SERVICES PVT. LTD.,21AABCS0923L1,BNOREAS000578/24,03-Dec-2024,No SAC Qty Discou Value\nnt Rate Amount Rate A...,541671.08,48750.40,48750.40,97500.80,Page 6,736672.68
2,SLV SECURITY SERVICES PVT. LTD.,21AABCS0923L1,BNOREAS000579/24,03-Dec-2024,No SAC Qty Discou Value\nnt Rate Amount Rate A...,164277.49,14784.97,14784.97,29569.94,Page 7,223417.37
3,SLV SECURITY SERVICES PVT. LTD.,09AABCS0923L1,BNUPNOD000736/24,04-Dec-2024,No SAC Qty Discou Value\nnt Rate Amount Rate A...,48208.79,4338.79,4338.79,8677.58,Page 8,65563.95
4,Ernst & Young LLP,09AAEFE1763C1ZQ,IN91UP5I002096,02 Dec 2024,Professional Fee for,1210250.00,0.00,0.00,217845.00,Page 24,1428095.00
5,Ernst & Young LLP,09AAEFE1763C1ZQ,IN91UP5I001780,04 Nov 2024,Fee for assistance in,412000.00,0.00,0.00,74160.00,Page 25,486160.00
6,Ernst & Young LLP,09AAEFE1763C1ZQ,IN91UP5I002557,06 Jan 2025,Fee for assistance in,412000.00,0.00,0.00,74160.00,Page 26,486160.00
7,Ernst & Young LLP,09AAEFE1763C1ZQ,IN91UP5I002102,02 Dec 2024,Fee for assistance in,412000.00,0.00,0.00,74160.00,Page 27,486160.00
8,Ernst & Young LLP,09AAEFE1763C1ZQ,IN91UP5I001515,01 Oct 2024,Fee for assistance in,412000.00,0.00,0.00,74160.00,Page 28,486160.00
9,SLV SECURITY SERVICES PVT. LTD.,21AABCS0923L1,BNOREAS000659/24,03-Jan-2025,No SAC Qty Discou Value\nnt Rate Amount Rate A...,541671.08,48750.40,48750.40,97500.80,Page 50,736672.68
